In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os

In [0]:
os.listdir('./')

['.config', 'drive', 'sample_data']

In [0]:
os.chdir('./drive/My Drive/drive/舆情监控之情感分析')

In [0]:
os.listdir('./')

['Untitled0.ipynb',
 'ratings_1_train.csv',
 'ratings_1_valid.csv',
 'ratings_2_valid.csv',
 'ratings_2_train.csv',
 'ratings_3_train.csv',
 'ratings_3_valid.csv',
 'ratings_4_valid.csv',
 'ratings_4_train.csv',
 '情感分析.ipynb',
 '情感分析1.ipynb',
 'ratings_train.csv',
 'ratings_valid.csv',
 'qingganfenxi-model.pt',
 '情感分析2.ipynb']

In [0]:
import torch.nn as nn
import torch

class RNN(nn.Module):
  
  def __init__(self, vocab_size, embedding_dim, padding_idx, hidden_dim, num_layers, bidirectional, dropout, output_dim):
    super(RNN,self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
    self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout)
    self.dropout = nn.Dropout(dropout)
    self.decoder = nn.Linear(hidden_dim*2, output_dim)
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers
    self.output_dim = output_dim

  def forward(self, text, hidden):
    #embedded=[sentence_len,batch_size,embedding_dim]
    embedded = self.dropout(self.embedding(text))
    #output=[sentence_len,batch_size,hidden_dim*num_directional]
    #hidden=[num_layers*num_directional,batch_size,hidden_dim] 
    #cell=[num_layers*num_directional,batch_size,hidden_dim] 
    output, (hidden, cell) = self.rnn(embedded, hidden)
    decoded = self.decoder(self.dropout(torch.cat((hidden[-2,:,:],hidden[-1,:,:]),dim=1)))
    return decoded, (hidden, cell)

  def init_hidden(self, batch_size, requires_grad=True):
    weight = next(self.parameters())
    return (
        weight.new_zeros((self.num_layers*2, batch_size, hidden_dim),requires_grad=requires_grad),
        weight.new_zeros((self.num_layers*2, batch_size, hidden_dim),requires_grad=requires_grad)
    )

In [0]:
vocab_size = 8167
embedding_dim = 100 
padding_idx = 1
hidden_dim = 128
num_layers = 2
bidirectional = True
dropout = 0.5
output_dim = 2

In [0]:
model = RNN(vocab_size, embedding_dim, padding_idx, hidden_dim, num_layers, bidirectional, dropout, output_dim)

In [0]:
import numpy as np

In [0]:
stoi = np.load('stoi.npy',allow_pickle=True).item()

In [0]:
stoi['我']

13

In [0]:
import torch
import torch.nn.functional as F
def predict_sentiment(sentence):
  device = torch.device('cuda')
  # model = TheModelClass(*args, **kwargs)
  model.load_state_dict(torch.load('./qingganfenxi-model.pt'))
  model.to(device)
  hidden = model.init_hidden(1)
  tokenized = [tok for tok in sentence]
  indexed = [stoi[t] for t in tokenized]
  tensor = torch.LongTensor(indexed).to(device)
  tensor = tensor.unsqueeze(1)
  output, hidden = model(tensor, hidden)
  output = output[0]
  
  prediction = F.softmax(output,dim=0)
  return prediction
    # return output

In [0]:
predict_sentiment('不错的一家餐厅，在香格里拉的一楼。\r\r\n座位比较少，很安静。\r\r\n食物也蛮新鲜的。\r\r\n或许和就餐时的心情有关，东西也觉得很棒。\r\r\n总的来说不错啦，有机会还会去的。\r\r\n')

tensor([0.9985, 0.0015], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [0]:
predict_sentiment('接电话的小姐态度很不好，我问他又没有教小孩的初级班，没有你就说没有，她说4个人可以开班，你自己找人去。我说你办培训班，我去找人，我上哪儿去找人?实在是太搞笑了。不知道老板是怎么想的，找这样的人来接电话，一天不知道给他得罪多少客人，连接电话的基本规矩都不懂，好吧，本来还想带孩子去学滑冰的，现在肯定不会再在这里学了，而且我也会在我的圈子里说不好的了。\r\r\n')

tensor([0.0027, 0.9973], device='cuda:0', grad_fn=<SoftmaxBackward>)